In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import activations

In [2]:
(train,test,validation),info = tfds.load(
    'cassava',
    split=['train','test','validation'],
    # data_dir='data'
    shuffle_files=True,
    batch_size=-1,
    as_supervised=True,
    with_info=True)

In [3]:
info

tfds.core.DatasetInfo(
    name='cassava',
    full_name='cassava/0.1.0',
    description="""
    Cassava consists of leaf images for the cassava plant depicting healthy and
    four (4) disease conditions; Cassava Mosaic Disease (CMD), Cassava Bacterial
    Blight (CBB), Cassava Greem Mite (CGM) and Cassava Brown Streak Disease (CBSD).
    Dataset consists of a total of 9430 labelled images.
    The 9430 labelled images are split into a training set (5656), a test set(1885)
    and a validation set (1889). The number of images per class are unbalanced with
    the two disease classes CMD and CBSD having 72% of the images.
    """,
    homepage='https://www.kaggle.com/c/cassava-disease/overview',
    data_path='C:\\Users\\fozil\\tensorflow_datasets\\cassava\\0.1.0',
    file_format=tfrecord,
    download_size=1.26 GiB,
    dataset_size=1.26 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string)

In [3]:
x_train,y_train = tfds.as_numpy(train)
x_test,y_test = tfds.as_numpy(test)
x_val,y_val = tfds.as_numpy(validation)

In [37]:
x_train[0].shape

(224, 224, 3)

In [6]:
def dataset_maker(img_data,labels):
    images = []
    for img in img_data:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,dsize=(224,224),interpolation=cv2.INTER_AREA)
        img = img.astype('float32')
        img /= 255
        images.append(img)
    images = np.array(images)
    labels = labels.astype('uint32')
    return images,labels

In [7]:
x_train,y_train = dataset_maker(x_train,y_train)
x_test,y_test = dataset_maker(x_test,y_test)
x_val,y_val = dataset_maker(x_val,y_val)

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(32)

## VGG19 from scratch

In [8]:
class ModelMine(tf.keras.Model):
    def __init__(self):
        super(ModelMine,self).__init__()

        self.conv1_1 = keras.layers.Conv2D(64,(3,3),padding = 'same',activation=activations.relu)
        self.conv1_2 = keras.layers.Conv2D(64,(3,3),padding = 'same',activation=activations.softmax)
        self.pool1 = keras.layers.MaxPooling2D((2,2),strides=(2,2))

        self.conv2_1 = keras.layers.Conv2D(128,(3,3),padding = 'same',activation=activations.relu)
        self.conv2_2 = keras.layers.Conv2D(128,(3,3),padding = 'same',activation=activations.relu)
        self.pool2 = keras.layers.MaxPooling2D((2,2),strides=(2,2))

        self.conv3_1 = keras.layers.Conv2D(256,(3,3),padding = 'same',activation=activations.relu)
        self.conv3_2 = keras.layers.Conv2D(256,(3,3),padding = 'same',activation=activations.relu)
        self.conv3_3 = keras.layers.Conv2D(256,(3,3),padding = 'same',activation=activations.relu)
        self.conv3_4 = keras.layers.Conv2D(256,(3,3),padding = 'same',activation=activations.relu)
        self.pool3 = keras.layers.MaxPooling2D((2,2),strides=(2,2))

        self.conv4_1 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.conv4_2 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.conv4_3 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.conv4_4 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.pool4 = keras.layers.MaxPooling2D((2,2),strides=(2,2))

        self.conv5_1 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.conv5_2 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.conv5_3 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.conv5_4 = keras.layers.Conv2D(512,(3,3),padding = 'same',activation=activations.relu)
        self.pool5 = keras.layers.MaxPooling2D((2,2),strides=(2,2))

        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(4096,activation=activations.relu)
        self.fc2 = keras.layers.Dense(4096,activation=activations.relu)
        self.fc3 = keras.layers.Dense(1000,activation=activations.relu)
        self.fc4 = keras.layers.Dense(100,activation=activations.relu)
        self.fc5 = keras.layers.Dense(5,activation=activations.softmax)

    def call(self,x):

        x = self.conv1_1(x) # 
        x = self.conv1_2(x) # 
        x = self.pool1(x)   #

        x = self.conv2_1(x) #
        x = self.conv2_2(x) #
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.conv3_4(x)
        x = self.pool3(x)

        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        x = self.conv4_4(x)
        x = self.pool4(x)

        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.conv5_3(x)
        x = self.conv5_4(x)
        x = self.pool5(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)

        return self.fc5(x)

In [9]:
model = ModelMine()

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [11]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [12]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [13]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [14]:
EPOCHS = 5
with tf.device('/device:GPU:0'):
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

C:\Users\fozil\miniconda3\envs\my_gpu\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Loss: 6.95292854309082, Accuracy: 46.516971588134766, Test Loss: 1.3443787097930908, Test Accuracy: 47.00265121459961
Epoch 2, Loss: 1.3517769575119019, Accuracy: 46.99434280395508, Test Loss: 1.345324993133545, Test Accuracy: 47.00265121459961
Epoch 3, Loss: 1.3472888469696045, Accuracy: 46.99434280395508, Test Loss: 1.345323085784912, Test Accuracy: 47.00265121459961
Epoch 4, Loss: 1.346398949623108, Accuracy: 46.99434280395508, Test Loss: 1.351625919342041, Test Accuracy: 47.00265121459961
Epoch 5, Loss: 1.3475369215011597, Accuracy: 46.99434280395508, Test Loss: 1.3436042070388794, Test Accuracy: 47.00265121459961


## Transfer learning with VGG19

In [9]:
vgg19 = keras.applications.vgg19.VGG19(input_shape=(224,224,3),include_top=False)

In [10]:
vgg19.trainable=False

In [11]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
vgg19_model = Sequential([
    vgg19,
    keras.layers.Flatten(),
    keras.layers.Dense(4096,activation=activations.relu),
    keras.layers.Dense(2048,activation=activations.relu),
    keras.layers.Dense(1000,activation=activations.relu),
    keras.layers.Dense(100,activation=activations.relu),
    keras.layers.Dense(5,activation=activations.softmax)])

In [13]:
vgg19_model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                 metrics=keras.metrics.SparseCategoricalAccuracy())

In [15]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='saved_model/model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]
with tf.device('/device:GPU:0'):
    vgg19_model.fit(
        train_ds,
        epochs=5,
        callbacks=my_callbacks,
        verbose = 1,
        validation_data = test_ds)

Epoch 1/5
177/177 [==============================] - 28s 109ms/step - loss: 1.7459 - sparse_categorical_accuracy: 0.5334 - val_loss: 1.1777 - val_sparse_categorical_accuracy: 0.5767
Epoch 2/5
177/177 [==============================] - 16s 90ms/step - loss: 1.0418 - sparse_categorical_accuracy: 0.6110 - val_loss: 1.1417 - val_sparse_categorical_accuracy: 0.5798
Epoch 3/5
177/177 [==============================] - 16s 89ms/step - loss: 0.9870 - sparse_categorical_accuracy: 0.6399 - val_loss: 1.1091 - val_sparse_categorical_accuracy: 0.6053
Epoch 4/5
177/177 [==============================] - 16s 90ms/step - loss: 0.9280 - sparse_categorical_accuracy: 0.6581 - val_loss: 1.3486 - val_sparse_categorical_accuracy: 0.5066
Epoch 5/5
177/177 [==============================] - 16s 90ms/step - loss: 0.9140 - sparse_categorical_accuracy: 0.6673 - val_loss: 1.2506 - val_sparse_categorical_accuracy: 0.5326


In [16]:
vgg19_model.evaluate(test_ds)

59/59 [==============================] - 3s 46ms/step - loss: 1.2506 - sparse_categorical_accuracy: 0.5326


[1.2506476640701294, 0.5326259732246399]